In [30]:
import pandas as pd
import numpy as np
import operator 
import re
import gc
import keras
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [5]:
! pwd

/Users/kate/Desktop/Hotels_reviews


In [6]:
embeddings_index = {}
f = open('./glove.840B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs
f.close()

print('GloVe data loaded')


GloVe data loaded


In [7]:
reviews_515 = pd.read_csv('./data/Hotel_Reviews.csv') 
reviews_515['reviews.text'] = reviews_515['Negative_Review']+reviews_515['Positive_Review']
reviews_515 = reviews_515[['reviews.text', 'Reviewer_Score']]
reviews_515['reviews.text'] = reviews_515["reviews.text"].apply(lambda x: x.replace("No Negative", "").replace("No Positive", "").replace("Nothing", ""))
reviews_515['reviews.rating'] = reviews_515['Reviewer_Score']/2
reviews_515.drop('Reviewer_Score',1,inplace=True)
reviews_515.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 2 columns):
reviews.text      515738 non-null object
reviews.rating    515738 non-null float64
dtypes: float64(1), object(1)
memory usage: 7.9+ MB


In [8]:
reviews_8k = pd.read_csv('./data/7282_1.csv') 
reviews_8k['reviews.rating']=reviews_8k['reviews.rating']/2
reviews_8k.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35912 entries, 0 to 35911
Data columns (total 19 columns):
address                 35912 non-null object
categories              35912 non-null object
city                    35912 non-null object
country                 35912 non-null object
latitude                35826 non-null float64
longitude               35826 non-null float64
name                    35912 non-null object
postalCode              35857 non-null object
province                35912 non-null object
reviews.date            35653 non-null object
reviews.dateAdded       35912 non-null object
reviews.doRecommend     0 non-null float64
reviews.id              0 non-null float64
reviews.rating          35050 non-null float64
reviews.text            35890 non-null object
reviews.title           34290 non-null object
reviews.userCity        16263 non-null object
reviews.username        35869 non-null object
reviews.userProvince    17518 non-null object
dtypes: float64(5), 

In [9]:
reviews_datafinity = pd.read_csv('./data/Datafiniti_Hotel_Reviews.csv')
reviews_datafinity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
id                      10000 non-null object
dateAdded               10000 non-null object
dateUpdated             10000 non-null object
address                 10000 non-null object
categories              10000 non-null object
primaryCategories       10000 non-null object
city                    10000 non-null object
country                 10000 non-null object
keys                    10000 non-null object
latitude                10000 non-null float64
longitude               10000 non-null float64
name                    10000 non-null object
postalCode              10000 non-null object
province                10000 non-null object
reviews.date            10000 non-null object
reviews.dateSeen        10000 non-null object
reviews.rating          10000 non-null float64
reviews.sourceURLs      10000 non-null object
reviews.text            9999 non-null object
reviews.title   

In [10]:
reviews = pd.read_csv('./data/reviews.csv', engine='python' )
reviews['review_text'] = reviews['review_text'].fillna(reviews['review_title'])
reviews = reviews.rename(columns = {'rating_value':'reviews.rating','review_text':'reviews.text'})
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8534 entries, 0 to 8533
Data columns (total 5 columns):
Unnamed: 0        8534 non-null int64
reviews.rating    8534 non-null float64
reviews.text      8534 non-null object
review_title      8528 non-null object
Hotels            8534 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 333.4+ KB


In [11]:
reviews_tr1=pd.read_csv('./data/kl_reviews_first_5.csv', engine='python' )
reviews_tr1 = reviews_tr1.drop(['Unnamed: 0', 'review_link', 'review_hotel','review_date', 'stay_date', 'review_title'], 1)
reviews_tr1 = reviews_tr1.rename(columns = {'review_rating':'reviews.rating','review_text':'reviews.text'})
reviews_tr1 = reviews_tr1.dropna()
print(reviews_tr1.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1694 entries, 0 to 1694
Data columns (total 2 columns):
reviews.rating    1694 non-null float64
reviews.text      1694 non-null object
dtypes: float64(1), object(1)
memory usage: 39.7+ KB
None


In [12]:
reviews_tr2=pd.read_csv('./data/kl_reviews_last_5.csv', engine='python' )
reviews_tr2 = reviews_tr2.drop(['Unnamed: 0', 'review_link', 'review_hotel','review_date', 'stay_date', 'review_title'], 1)
reviews_tr2 = reviews_tr2.rename(columns = {'review_rating':'reviews.rating','review_text':'reviews.text'})
reviews_tr2 = reviews_tr2.dropna()
reviews_tr2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 781 entries, 1 to 1039
Data columns (total 2 columns):
reviews.rating    781 non-null float64
reviews.text      781 non-null object
dtypes: float64(1), object(1)
memory usage: 18.3+ KB


In [13]:
reviews_total = pd.concat([ 
                           reviews[['reviews.rating', 'reviews.text']],
                           reviews_8k[['reviews.rating', 'reviews.text']],
                           reviews_datafinity[['reviews.rating', 'reviews.text']],
                           reviews_515[['reviews.rating', 'reviews.text']],
                           reviews_tr1, reviews_tr2
                          ],
                          axis=0)


reviews_total.reset_index(drop=True, inplace=True)
reviews_total['reviews.rating']=reviews_total['reviews.rating'].apply(lambda x: round(x,1))

In [14]:
reviews_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572659 entries, 0 to 572658
Data columns (total 2 columns):
reviews.rating    571797 non-null float64
reviews.text      572636 non-null object
dtypes: float64(1), object(1)
memory usage: 8.7+ MB


In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

def NLTK_sentiment_analysis(text):

    sid = SentimentIntensityAnalyzer()
    return sid.polarity_scores(text)['compound']
  
reviews_total['reviews.text'] = reviews_total['reviews.text'].astype('str')  
# reviews_total['polarity'] = reviews_total['reviews.text'].apply(NLTK_sentiment_analysis)

/Users/kate/anaconda3/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kate/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
reviews_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572659 entries, 0 to 572658
Data columns (total 2 columns):
reviews.rating    571797 non-null float64
reviews.text      572659 non-null object
dtypes: float64(1), object(1)
memory usage: 8.7+ MB


In [17]:
reviews_total["reviews.rating"] = reviews_total["reviews.rating"].apply(lambda x: 1 if x > 2 else 0)

In [18]:
reviews_total['reviews.rating'].value_counts()

1    537248
0     35411
Name: reviews.rating, dtype: int64

In [19]:
positive_train = reviews_total.loc[reviews_total['reviews.rating']==1]
negative_train = reviews_total.loc[reviews_total['reviews.rating']==0]

In [20]:
from sklearn.utils import resample

negative_train = resample(negative_train, n_samples = 500000, replace=True, random_state=123)

In [21]:
reviews_total = pd.concat([positive_train,negative_train ], 0)
reviews_total = reviews_total.dropna()

In [22]:
reviews_total["reviews.rating"].value_counts()

1    537248
0    500000
Name: reviews.rating, dtype: int64

In [23]:
reviews_total[:20]

,reviews.rating,reviews.text
0,1,One of the best luxury properties in Singapore...
1,1,We love doing staycations here. Great location...
2,1,Expensive but nice place <br/>
3,1,It was a truly relaxing experience. The ambien...
4,1,Very relaxing. With very good breakfast <br/>
5,1,The main bar was closed for a private party (w...
6,1,"Pros: Excellent and attentive service, from kn..."
7,1,"Customer service was very busy and hence, ther..."
8,1,Not my first stay here and would definitely co...
9,1,All is perfect from room to services and great...


In [24]:
reviews_for_training = reviews_total.copy(deep=True)

In [25]:
label = reviews_for_training['reviews.rating']
train_set = reviews_for_training.drop('reviews.rating',1)


In [26]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))

import re



## Iterate over the data to preprocess by removing stopwords
lines_without_stopwords=[] 
for line in train_set['reviews.text'].values: 
    line = line.lower()
    line_by_words = re.findall(r'(?:\w+)', line, flags = re.UNICODE) # remove punctuation ans split
    new_line=[]
    for word in line_by_words:
        word = ''.join([i for i in word if not i.isdigit()])
        if word not in stop and len(word)>2:
            new_line.append(word)
    lines_without_stopwords.append(new_line)
texts = lines_without_stopwords

print(texts[0:5]) 


[nltk_data] Downloading package stopwords to /Users/kate/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[['one', 'best', 'luxury', 'properties', 'singapore', 'absolute', 'bliss', 'peaceful', 'sea', 'views'], ['love', 'staycations', 'great', 'location', 'sentosa', 'right', 'near', 'beach', 'importantly', 'pet', 'friendly', 'able', 'enjoy', 'experience', 'dog'], ['expensive', 'nice', 'place'], ['truly', 'relaxing', 'experience', 'ambience', 'deco', 'room', 'exceptional', 'great', 'recharging', 'soul', 'like', 'complimentary', 'tea', 'time', 'setting', 'however', 'taste', 'cakes', 'desserts', 'improved', 'good', 'shuttle', 'bus', 'extended', 'hours', 'instead', 'current', 'sofitel', 'services', 'beyond'], ['relaxing', 'good', 'breakfast']]


In [27]:
texts[20:25]

[['anniversary',
  'staycation',
  'appreciate',
  'room',
  'upgraded',
  'hotel',
  'kind',
  'gift',
  'bottle',
  'wine',
  'cons',
  'kids',
  'friendly',
  'hotel',
  'pool',
  'weekend',
  'swarmed',
  'children',
  'also',
  'hotel',
  'running',
  'occupancy',
  'asked',
  'staff',
  'enough',
  'sun',
  'beds',
  'around'],
 ['place',
  'pleasant',
  'good',
  'resort',
  'stay',
  'improvement',
  'needs',
  'done'],
 ['time',
  'coming',
  'capella',
  'feel',
  'time',
  'standard',
  'expected',
  'previous',
  'staffs',
  'friendly',
  'without',
  'asking',
  'things',
  'done',
  'perfectly',
  'waiting',
  'villa',
  'ready',
  'lobby',
  'refreshments',
  'table',
  'cleared',
  'manager',
  'came',
  'ask',
  'serve',
  'second',
  'round',
  'refreshments',
  'told',
  'first',
  'even',
  'belong',
  'stay',
  'villa',
  'previous',
  'times',
  'need',
  'call',
  'buggy',
  'arrives',
  'within',
  'mins',
  'time',
  'call',
  'times',
  'without',
  'confirmin

In [28]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


MAX_NUM_WORDS = 1000
MAX_SEQUENCE_LENGTH = 100
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(data[:5])

labels = to_categorical(np.asarray(label))
print(data.shape)
print(labels.shape)

# X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=42, test_size=0.2)

Found 94411 unique tokens.
[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  15  96 933 433]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0 415   7
    4  93 111 314 739  13 295 526  99 681]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

In [31]:
with open('tokenizer1.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [32]:
# data = pd.DataFrame(data)
# print(data.head())
# print(data.shape)


In [33]:
# train_set = train_set.reset_index(drop=True)
# data['polarity_score']= train_set['polarity']

In [34]:
from keras.layers import Embedding
from keras.initializers import Constant

## EMBEDDING_DIM =  ## seems to need to match the embeddings_index dimension
EMBEDDING_DIM = embeddings_index.get('a').shape[0]
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [35]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation

model = Sequential()
model.add(Embedding(num_words, 300, input_length=100, weights= [embedding_matrix], trainable=False))

model.add(Dropout(0.1))
model.add(Conv1D(32, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(100))
model.add(Dense(2, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [36]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, random_state=42, test_size=0.2)

In [37]:
print(labels.shape)
model.fit(X_train, y_train, validation_split=0.1, epochs = 5)

(1037248, 2)
Train on 746818 samples, validate on 82980 samples
Epoch 1/5
746818/746818 [==============================] - 1995s 3ms/step - loss: 0.3487 - acc: 0.8471 - val_loss: 0.3046 - val_acc: 0.8719
Epoch 2/5
746818/746818 [==============================] - 2130s 3ms/step - loss: 0.2917 - acc: 0.8765 - val_loss: 0.2705 - val_acc: 0.8881
Epoch 3/5
746818/746818 [==============================] - 2636s 4ms/step - loss: 0.2676 - acc: 0.8887 - val_loss: 0.2562 - val_acc: 0.8954
Epoch 4/5
746818/746818 [==============================] - 2199s 3ms/step - loss: 0.2549 - acc: 0.8945 - val_loss: 0.2480 - val_acc: 0.8993
Epoch 5/5
746818/746818 [==============================] - 2223s 3ms/step - loss: 0.2478 - acc: 0.8980 - val_loss: 0.2446 - val_acc: 0.9012


In [38]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, roc_auc_score 

pred = model.predict(X_test)
pred = np.round(pred)
print('Acc:',accuracy_score(y_test, pred))
print(roc_auc_score(y_test, pred))
print(confusion_matrix(y_test.argmax(axis=1), pred.argmax(axis=1)))


Acc: 0.9018944323933478
0.9025843484891444
[[92608  7681]
 [12671 94490]]


In [39]:
from sklearn.metrics import classification_report
print(classification_report(y_test, pred))

             precision    recall  f1-score   support

          0       0.88      0.92      0.90    100289
          1       0.92      0.88      0.90    107161

avg / total       0.90      0.90      0.90    207450



In [40]:
import pickle
filename = 'Sentiment_model_1_2.pkl'
pickle.dump(model, open(filename, 'wb'))